In [19]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [22]:
df["sex"].value_counts()

sex
Male      157
Female     87
Name: count, dtype: int64

In [23]:
df["smoker"].value_counts()

smoker
No     151
Yes     93
Name: count, dtype: int64

In [24]:
df["day"].value_counts()

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

In [25]:
df["time"].value_counts()

time
Dinner    176
Lunch      68
Name: count, dtype: int64

In [26]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [27]:
# independent and dependent features
x = df[['tip', 'sex', 'smoker', 'day', 'time', 'size']]
y = df["total_bill"]

In [28]:
x.head()

,tip,sex,smoker,day,time,size
0,1.01,Female,No,Sun,Dinner,2
1,1.66,Male,No,Sun,Dinner,3
2,3.50,Male,No,Sun,Dinner,3
3,3.31,Male,No,Sun,Dinner,2
4,3.61,Female,No,Sun,Dinner,4


In [29]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=42)

## Feature Encoding

#### Label encoding 

In [30]:
from sklearn.preprocessing import LabelEncoder
lab1 = LabelEncoder()
lab2 = LabelEncoder()
lab3 = LabelEncoder()

In [31]:
import warnings
warnings.filterwarnings('ignore')
x_train['sex'] = lab1.fit_transform(x_train['sex'])
x_train['smoker'] = lab2.fit_transform(x_train['smoker'])
x_train['time'] = lab3.fit_transform(x_train['time'])

In [32]:
x_test['sex'] = lab1.transform(x_test['sex'])
x_test['smoker'] = lab2.transform(x_test['smoker'])
x_test['time'] = lab3.transform(x_test['time'])

In [33]:
x_test.head()

,tip,sex,smoker,day,time,size
24,3.18,1,0,Sat,0,2
6,2.00,1,0,Sun,0,2
153,2.00,1,0,Sun,0,4
211,5.16,1,1,Sat,0,4
198,2.00,0,1,Thur,1,2


#### OneHot encoding --- column transform

In [34]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [35]:
ct = ColumnTransformer(transformers=[('onehot',OneHotEncoder(drop='first'),[3])],remainder='passthrough')

In [38]:
import sys
np.set_printoptions(threshold = sys.maxsize)
x_train = ct.fit_transform(x_train)

In [39]:
x_test = ct.transform(x_test)

### SVR - Support Vector Regression

In [40]:
from sklearn.svm import SVR
svr = SVR()

In [41]:
svr.fit(x_train,y_train)

SVR()

In [42]:
y_pred = svr.predict(x_test)

In [43]:
from sklearn.metrics import r2_score,mean_absolute_error
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

0.49798620106004743
4.463296539661224


## Hyperparameter tuning using GridSearchCV

In [44]:
# defining the parameter range
param_grid = {'C':[0.1,1,10,100,1000],
             'gamma':[1,0.1,0.01,0.001],
             'kernel':['rbf']}

In [45]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(SVR(),param_grid=param_grid,cv=5,refit=True,verbose=3)

In [46]:
grid.fit(x_train,y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.114 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.008 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.003 total time=   0.0s
[CV 4/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.015 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.123 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.019 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.096 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.078 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.099 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-0.020 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=0.01, kernel=rbf;, score=-0.102 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf']},
             verbose=3)

In [49]:
y_prediction = grid.predict(x_test)

In [50]:
grid.best_params_

{'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}

In [51]:
from sklearn.metrics import r2_score,mean_absolute_error
print(r2_score(y_test,y_prediction))
print(mean_absolute_error(y_test,y_prediction))

0.5630727637616448
4.2604839738722715
